In [18]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd
import requests
import time
import lxml

In [19]:
url = 'http://opendata.burtgel.gov.mn/lesinfo/'

url_link_check = [f"{url}{i}" for i in range(1000000, 9999999)]
print(len(url_link_check))

8999999


In [20]:
registr = pd.read_csv('data_company.csv')

url_link_use = []
for i in registr['Регистерийн дугаар']:
    url_link_use.append(f'{url}{i}')
print(len(url_link_use))


6566


In [21]:
print(len(url_link_check)-len(url_link_use))

8993433


In [22]:
url_link = list(set(url_link_check) - set(url_link_use))
print(len(url_link))

8993861


In [23]:
headers =['Регистр', 'Нэр', 'Хаяг', 'Бүртгэсэн_огноо', 'Үйл_ажиллагааны_чиглэл',
          'Хувьцаа_эзэмшигчийн_тоо', 'Хувьцаа_эзэмшигч_гишүүд', 
          'Итгэмжлэлгүйгээр_төлөөлөх_албан_тушаалтан','ИТАТ_бүртгэсэн_огноо',
          'Эцсийн_Өмчлөгч', 'ЭӨ_бүртгэсэн_огноо']


data = pd.DataFrame(columns=headers)
data

,Регистр,Нэр,Хаяг,Бүртгэсэн_огноо,Үйл_ажиллагааны_чиглэл,Хувьцаа_эзэмшигчийн_тоо,Хувьцаа_эзэмшигч_гишүүд,Итгэмжлэлгүйгээр_төлөөлөх_албан_тушаалтан,ИТАТ_бүртгэсэн_огноо,Эцсийн_Өмчлөгч,ЭӨ_бүртгэсэн_огноо


In [24]:
def crawl(url):    
    ua = UserAgent()
    fake_ua = {'user-agent': ua.random}

    used_link = []

    
    response = requests.get(url=url, headers=fake_ua)
    response.encoding = 'utf-8'

    soup = BeautifulSoup(response.text, 'lxml')

    table = soup.find('table', class_='table table-bordered table-hover').find('tbody')
    registr = table.find('td')

    if registr.text != "Тохирох мэдээлэл олдсонгүй":
        try:
            row = []

            table = soup.find('table', class_='table table-bordered table-hover').find('tbody')

            # регистр
            try:
                regist = table.find('td')
                row.append(regist.text)
            except Exception as e:
                row.append('')

            # Нэр
            try:
                name = table.find_all('td')[1]
                row.append(name.text)
            except Exception as e:
                row.append('')

            # Хаяг
            try:
                adress = table.find_all('td')[6]
                row.append(adress.text)
            except Exception as e:
                row.append('')

            # Бүртгүүлсэн огноо
            try:
                date = table.find_all('td')[2]
                row.append(date.text)
            except Exception as e:
                row.append('')

            # Үйл ажиллагааны чиглэл
            try:
                table3 = soup.find_all('table', class_='table table-bordered table-hover')[3]
                field_act = ""
                field = table3.find_all('tr')
                for i in field[1:]:
                    field_act = f'{i.find_all("td")[3].text}, {field_act}'
                row.append(field_act)
            except Exception as e:
                row.append('')

            # Хувьцаа эзэмшигчийн тоо
            try:
                count = table.find_all('td')[5]
                row.append(count.text)
            except Exception as e:
                row.append('')

            # Хувьцаа эзэмшигч гишүүд
            try:
                table1 = soup.find_all('table', class_='table table-bordered table-hover')[1]
                sur_name = ""
                table1 = table1.find_all('tr')
                for i in table1[1:]:
                    surname = i.find_all("td")[3].text + '.' + i.find_all('td')[4].text
                    sur_name = f'{surname}, {sur_name}'
                row.append(sur_name)
            except Exception as e:
                row.append('')

            # Итгэмжлэлгүйгээр төлөөлөх албан тушаалтан
            try:
                table2 = soup.find_all('table', class_='table table-bordered table-hover')[2]
                s = table2.find_all('td')[3].text
                n = table2.find_all('td')[4].text
                row.append(s + '.' + n)
            except Exception as e:
                row.append('')

            # ИТАТ_бүртгэсэн_огноо
            try:
                table2 = soup.find_all('table', class_='table table-bordered table-hover')[2]
                t = table2.find_all('td')[5].text
                row.append(t)
            except Exception as e:
                row.append('')

            # Эцсийн өмчлөгч
            try:
                table4 = soup.find_all('table', class_='table table-bordered table-hover')[4]
                sur_name1 = ""
                table4 = table4.find_all('tr')
                for i in table4[1:]:
                    surname1 = i.find_all("td")[2].text + '.' + i.find_all('td')[3].text
                    sur_name1 = f'{surname1}, {sur_name1}'
                row.append(sur_name1)
            except Exception as e:
                row.append('')

            # Эцсийн өмчлөгч бүртгэсэн огноо
            try:
                table4 = soup.find_all('table', class_='table table-bordered table-hover')[4]
                tt = table4.find_all('td')[4].text
                row.append(tt)
            except Exception as e:
                row.append('')
        except Exception as e:
            pass
    return row

In [8]:
import threading


,Регистр,Нэр,Хаяг,Бүртгэсэн_огноо,Үйл_ажиллагааны_чиглэл,Хувьцаа_эзэмшигчийн_тоо,Хувьцаа_эзэмшигч_гишүүд,Итгэмжлэлгүйгээр_төлөөлөх_албан_тушаалтан,ИТАТ_бүртгэсэн_огноо,Эцсийн_Өмчлөгч,ЭӨ_бүртгэсэн_огноо


In [12]:
def batch(lists, size):
    batch_list = []
    n = len(lists) // size
    for i in range(n):
        batch_list.append(lists[i * size: size + i * size])
        
    return batch_list



In [17]:
batch(url_link[0:50], 20)

[['http://opendata.burtgel.gov.mn/lesinfo/1320791',
  'http://opendata.burtgel.gov.mn/lesinfo/7852387',
  'http://opendata.burtgel.gov.mn/lesinfo/1588399',
  'http://opendata.burtgel.gov.mn/lesinfo/9418624',
  'http://opendata.burtgel.gov.mn/lesinfo/2519822',
  'http://opendata.burtgel.gov.mn/lesinfo/5832244',
  'http://opendata.burtgel.gov.mn/lesinfo/9549176',
  'http://opendata.burtgel.gov.mn/lesinfo/3846211',
  'http://opendata.burtgel.gov.mn/lesinfo/1681938',
  'http://opendata.burtgel.gov.mn/lesinfo/9981088',
  'http://opendata.burtgel.gov.mn/lesinfo/7178887',
  'http://opendata.burtgel.gov.mn/lesinfo/1127655',
  'http://opendata.burtgel.gov.mn/lesinfo/3923408',
  'http://opendata.burtgel.gov.mn/lesinfo/9380640',
  'http://opendata.burtgel.gov.mn/lesinfo/7565523',
  'http://opendata.burtgel.gov.mn/lesinfo/2090501',
  'http://opendata.burtgel.gov.mn/lesinfo/8796434',
  'http://opendata.burtgel.gov.mn/lesinfo/7318422',
  'http://opendata.burtgel.gov.mn/lesinfo/2853848',
  'http://op

In [26]:
a = [1]

len(a)

1